<a href="https://colab.research.google.com/github/Estocastic/Hello-world/blob/master/COD_BPN_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
directorio = '/content/drive/My Drive/Datos_BPN2023'

# CAPITULO 1. NACIMIENTOS

## PARTE 1. CARGA DE LOS DATOS NACIMIENTOS Y FILTROS LIMITROFES

### 1.1. CARGA INICIAL DE NACIMIENTOS Y MORTALIDAD EN LA CIUDAD DE CALI



#### CALI

In [ ]:
# Lista para almacenar los DataFrames de Excel
dfs = []

# Cargar los data frames de nacimientos
anyo_ini=2011
for filename in os.listdir(directorio):
    if filename.startswith("NACIM") and filename.endswith(('xls', 'xlsx')):  # Filtrar si el archivo es de nacimiento

        df = pd.read_excel(os.path.join(directorio, filename))
        df['anyo']=anyo_ini
        print(filename)
        dfs.append(df)
        anyo_ini=anyo_ini+1

In [ ]:
pd.DataFrame(list(map(lambda x: x.columns, dfs))).T # la variable fecha ultima modificacion no estan en la base 2017

In [6]:
dfs=list(map(lambda x: x.drop('Fecha Última Modificación', axis=1, errors='ignore'), dfs)) # se retira la variable

In [167]:
df_final = pd.concat(dfs, ignore_index=True)

#### VALLE

In [166]:
nac_valle = pd.read_excel('/content/drive/My Drive/Datos_BPN2023/BD_Nv_2011_2021_HABC_v2.xlsx')

#### FILTROS LIMITROFES: SE DESCARTAN NACIMIENTOS FUERA DEL VALLE

In [168]:
# Identificamos filtros: Nacimientos solo cuyo departamento de residencia fuese el valle.
(df_final['Departamento Residencia'] == 'VALLE DEL CAUCA').value_counts()

True     377416
False     24862
Name: Departamento Residencia, dtype: int64

In [169]:
# Identificamos filtros: Nacimientos solo cuyo departamento de residencia fuese el valle.
(nac_valle['CODPTORE'] == 76).value_counts()

True    265838
Name: CODPTORE, dtype: int64

In [69]:
f_valle_dptres = df_final['Departamento Residencia'] == 'VALLE DEL CAUCA'

In [170]:
df_final = df_final[f_valle_dptres].copy()

## PARTE 2. APLICACIÓN DE FILTROS PARA LLEGAR A LA POBLACIÓN OBJETIVO:

### 2.1. Construcción de los filtros



In [171]:
print('Médidas iniciales del Tiempo gestación y el Peso al nacer: \n')
print(f'CALI \n',df_final[['Tiempo Gestación','Peso']].describe().round())
print(f'VALLE \n',nac_valle[['T_GES','PESO_NAC']].describe().round())

Médidas iniciales del Tiempo gestación y el Peso al nacer: 

CALI 
        Tiempo Gestación      Peso
count          377343.0  377412.0
mean               38.0    3126.0
std                 2.0     556.0
min                18.0     220.0
25%                38.0    2860.0
50%                39.0    3180.0
75%                40.0    3480.0
max                49.0    5960.0
VALLE 
           T_GES  PESO_NAC
count  265838.0  265838.0
mean       39.0    3147.0
std         3.0     549.0
min        15.0     208.0
25%        38.0    2880.0
50%        39.0    3180.0
75%        40.0    3490.0
max        99.0    9999.0


In [172]:
f_bpnt_cali = ((df_final['Peso'] < 2500) & (df_final['Tiempo Gestación'] >= 37))
conteo = f_bpnt_cali.sum()
porcentaje = (conteo / len(df_final)) * 100

print(((df_final['Peso'] < 2500 ) & (df_final['Tiempo Gestación'] >= 37)).value_counts())
print(f'Número de nacidos vivos en Cali con BPNT: {conteo}')
print(f'Porcentaje de nacidos vivos en el Valle con BPNT: {porcentaje:.2f}%')


False    365640
True      11776
dtype: int64
Número de nacidos vivos en Cali con BPNT: 11776
Porcentaje de nacidos vivos en el Valle con BPNT: 3.12%


In [173]:
f_bpnt_valle = ((nac_valle['PESO_NAC'] < 2500 ) & (nac_valle['T_GES'] >= 37))
conteo = f_bpnt_valle.sum()
porcentaje = (conteo / len(df_final)) * 100

print(((nac_valle['PESO_NAC'] < 2500 ) & (nac_valle['T_GES'] >= 37)).value_counts())
print(f'Número de nacidos vivos en Cali con BPNT: {conteo}')
print(f'Porcentaje de nacidos vivos en el Valle con BPNT: {porcentaje:.2f}%')


False    258117
True       7721
dtype: int64
Número de nacidos vivos en Cali con BPNT: 7721
Porcentaje de nacidos vivos en el Valle con BPNT: 2.05%


### 2.2. Aplicación de los filtros: POBLACIÓN OBJETIVO

In [174]:
nac_cali = df_final[f_bpnt_cali].copy()
nac_valle = nac_valle[f_bpnt_valle].copy()

print(nac_cali.shape)
print(nac_valle.shape)

(11776, 51)
(7721, 45)


#### 2.2.1. PLUS: eliminar nacimientos fuera del valle

Conclusión: En total se tienen 11.776 + 7721 = 19.497
 registros de nacimientos con BPNT sumados en las dos fuentes de datos. Sin embargo, antes de unirlos falta corroborar y descartar de una de las bases registros con cédulas que estén contenidos en ambos y que correspondan al mismo nacimiento (fecha de nacimiento).

### 2.3. ADICIONAL:  Recategorización de las variables y Se lección de Columnas

Se recategorizan las variables del Valle del cauca con base en los valores asociados en df_mapeo que relaciona número y nombre, según el formulario de nacimientos RUAF.

Las de Cali no necesitan recategorización.

In [175]:
df_mapeo = pd.read_excel('/content/drive/My Drive/Datos_BPN2023/variables_codificar.xlsx')

In [176]:
mapeo_dict = {}
for columna in df_mapeo['name'].unique():
    sub_df = df_mapeo[df_mapeo['name'] == columna].copy()
    mapeo_dict[columna] = dict(zip(sub_df['numero'], sub_df['categoria']))
for columna, mapeo in mapeo_dict.items():
    nac_valle.loc[:,columna] = nac_valle.loc[:,columna].map(mapeo)

#### Se seleccionan ***`parcialmente`*** las variables de interés colocandoseles el mismo nombre para su posterior unión. Se mantienen los nombres de las columnas del conjunto de datos de Cali

In [177]:
nac_cali = nac_cali[['Municipio Nacimiento' , 'Area Nacimiento' , 'Sitio Parto' , 'IPS' , 'Sexo' , 'Peso' , 'Talla' , 'Fecha Nacimiento' , 'Parto Atendido Por' , 'Tiempo Gestación' , 'Número Consultas Prenatales' , 'Tipo Parto' , 'Multiplicidad Embarazo' , 'APGAR1' , 'APGAR5' , 'Grupo Sanguineo' , 'Factor RH' , 'Pertenencia Étnica' , 'Pueblo Indígena' , 'Tipo Documento Madre' , 'Numero Documento Madre' , 'Edad Madre' , 'Estado Conyugal Madre' , 'Último Año Estudios Madre' , 'Municipio Residencia' , 'Área Residencia' , 'Número Hijos Nacidos Vivos' , 'Fecha Nacimiento Anterior Hijo' , 'Numero Embarazos' , 'Régimen Seguridad Social' , 'EPS' , 'Edad Padre' , 'Último Año Estudios Padre']]

In [178]:
nac_valle=nac_valle[['COD_MUNIC' , 'AREANAC' , 'SIT_PARTO' , 'NOM_INST' , 'SEXO' , 'PESO_NAC' , 'TALLA_NAC' , 'FECHA_NAC' , 'ATEN_PAR' , 'T_GES' , 'NUMCONSUL' , 'TIPO_PARTO' , 'MUL_PARTO' , 'APGAR1' , 'APGAR2' , 'IDHEMOCLAS' , 'IDFACTORRH' , 'IDPERTET' , 'NOM_PUEB' , 'DOC_ID' , 'NRO_DOC' , 'EDAD_MADRE' , 'EST_CIVM' , 'NIV_EDUM' , 'CODMUNRE' , 'AREA_RES' , 'N_HIJOSV' , 'FECHA_NACM' , 'N_EMB' , 'SEG_SOCIAL' , 'NOMCLASAD' , 'EDAD_PADRE' , 'NIV_EDUP']]

In [179]:
nac_valle.columns = ['Municipio Nacimiento' , 'Area Nacimiento' , 'Sitio Parto' , 'IPS' , 'Sexo' , 'Peso' , 'Talla' , 'Fecha Nacimiento' , 'Parto Atendido Por' , 'Tiempo Gestación' , 'Número Consultas Prenatales' , 'Tipo Parto' , 'Multiplicidad Embarazo' , 'APGAR1' , 'APGAR5' , 'Grupo Sanguineo' , 'Factor RH' , 'Pertenencia Étnica' , 'Pueblo Indígena' , 'Tipo Documento Madre' , 'Numero Documento Madre' , 'Edad Madre' , 'Estado Conyugal Madre' , 'Último Año Estudios Madre' , 'Municipio Residencia' , 'Área Residencia' , 'Número Hijos Nacidos Vivos' , 'Fecha Nacimiento Anterior Hijo' , 'Numero Embarazos' , 'Régimen Seguridad Social' , 'EPS' , 'Edad Padre' , 'Último Año Estudios Padre']

In [180]:
pd.DataFrame({'Col_cali': nac_cali.columns, 'Col_valle': nac_valle.columns}).T

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
Col_cali,Municipio Nacimiento,Area Nacimiento,Sitio Parto,IPS,Sexo,Peso,Talla,Fecha Nacimiento,Parto Atendido Por,Tiempo Gestación,...,Último Año Estudios Madre,Municipio Residencia,Área Residencia,Número Hijos Nacidos Vivos,Fecha Nacimiento Anterior Hijo,Numero Embarazos,Régimen Seguridad Social,EPS,Edad Padre,Último Año Estudios Padre
Col_valle,Municipio Nacimiento,Area Nacimiento,Sitio Parto,IPS,Sexo,Peso,Talla,Fecha Nacimiento,Parto Atendido Por,Tiempo Gestación,...,Último Año Estudios Madre,Municipio Residencia,Área Residencia,Número Hijos Nacidos Vivos,Fecha Nacimiento Anterior Hijo,Numero Embarazos,Régimen Seguridad Social,EPS,Edad Padre,Último Año Estudios Padre


In [181]:
nac_cali.shape

(11776, 33)

## PARTE 3. UNIÓN DE LOS CONJUNTOS DE DATOS VALLE Y CALI EN UNO SOLO

3.4.2. APLICACIÓN DE LOS FILTROS:

Antes de unir los data sets, se elabora una comparación mediante las llaves de integración (Número de documento y fecha de nacimiento).
Para responder a la pregunta:

¿Cuáles de los registros corresponden a los mismos nacimientos?. Y teniendo como base la fuente de Cali, se eliminarán aquellos encontrados como repetidos de la fuente de Valle del cauca.


### 3.1. CONSTRUCCIÓN DE LOS FILTROS 2: CÉDULAS REDUNDANTES.

Se necesita unir los datos y valle en uno solo. pero antes, se corrobora que la unión no contenga datos que estén duplicados y/o que antes de hacer los filtros haya necesidad de mantener.

#### Subfiltro : Quítamos registros con número de identificación faltante.

In [182]:
((nac_cali['Numero Documento Madre'] == 99999999999)|
 (nac_cali['Numero Documento Madre'].isna())|
 (nac_cali['Numero Documento Madre'] == 0)).value_counts()

False    11725
True        51
Name: Numero Documento Madre, dtype: int64

In [183]:
((nac_valle['Numero Documento Madre'] == 99999999999)|
 (nac_valle['Numero Documento Madre'].isna())|
 (nac_valle['Numero Documento Madre'] == 0)).value_counts()

False    7682
True       39
Name: Numero Documento Madre, dtype: int64

In [184]:
f_idna_cali = ((nac_cali['Numero Documento Madre'] == 99999999999)|
 (nac_cali['Numero Documento Madre'].isna())|
 (nac_cali['Numero Documento Madre'] == 0))

f_idna_valle = ((nac_valle['Numero Documento Madre'] == 99999999999)|
 (nac_valle['Numero Documento Madre'].isna())|
 (nac_valle['Numero Documento Madre'] == 0))

In [185]:
nac_cali = nac_cali[~f_idna_cali].copy()
nac_valle = nac_valle[~f_idna_valle].copy()

#### Convertimos a formato fecha asegurandonos de que no hayan filtros con fechas vacías:

In [186]:
pd.to_datetime(nac_cali['Fecha Nacimiento'], errors = 'coerce').isna().value_counts()

False    11725
Name: Fecha Nacimiento, dtype: int64

In [187]:
pd.to_datetime(nac_valle['Fecha Nacimiento'], errors = 'coerce').isna().value_counts()

<ipython-input-187-bc17c2fd696c>:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  pd.to_datetime(nac_valle['Fecha Nacimiento'], errors = 'coerce').isna().value_counts()


False    7682
Name: Fecha Nacimiento, dtype: int64

In [188]:
# COMO TODAS LAS FECHAS ESTUVIERON BIEN CONVERTIDAS AHORA SE REEMPLAZAN LA MISMA FUNCIÓN EN EN CADA DATA FRAME:
nac_cali['Fecha Nacimiento'] = pd.to_datetime(nac_cali['Fecha Nacimiento'], errors = 'coerce')
nac_valle['Fecha Nacimiento'] = pd.to_datetime(nac_valle['Fecha Nacimiento'], errors = 'coerce')

<ipython-input-188-899dbef886de>:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  nac_valle['Fecha Nacimiento'] = pd.to_datetime(nac_valle['Fecha Nacimiento'], errors = 'coerce')


#### CRUCE DE NUMERO DE IDENTIFICACIÓN - VERIFICACIÓN DE DATOS REDUNDANTES.

Se localizan los índices de los registros duplicados en el conjunto de datos del Valle con respecto a Cali.
Elimina esos registros duplicados del conjunto de datos del Valle.

In [189]:
# VOLVEMOS COMPARABLES LAS ID
nac_cali['Numero Documento Madre'] = nac_cali['Numero Documento Madre'].astype(str)
nac_valle['Numero Documento Madre'] = nac_valle['Numero Documento Madre'].astype(int).astype(str)

In [190]:
nac_cali['Fecha Nacimiento']

42       2020-01-11
53       2020-01-15
57       2020-01-15
79       2020-01-09
116      2020-01-26
            ...    
402185   2013-02-01
402208   2013-02-11
402215   2013-05-13
402228   2013-07-03
402250   2013-09-05
Name: Fecha Nacimiento, Length: 11725, dtype: datetime64[ns]

In [191]:
nac_valle[['Numero Documento Madre', 'Fecha Nacimiento']].isin(nac_cali[['Numero Documento Madre', 'Fecha Nacimiento']]).all(axis=1).value_counts()

False    7682
dtype: int64

In [192]:
nac_cali[['Numero Documento Madre', 'Fecha Nacimiento']].isin(nac_valle[['Numero Documento Madre', 'Fecha Nacimiento']]).all(axis=1).value_counts()

False    11725
dtype: int64

No se encontraron duplicados. ❗ ⚠

In [110]:
'''ALTERNATIVA PARA HACERLO CON LA CREACIÓN DE LLAVES: AL CORRERAR EN UNA CELDA TEMPORAL DIO EL MISMO RESULTADO.

# Crear una nueva columna llamada 'llave' que concatena 'ID' y 'fecha_nacimiento' en ambos DataFrames
nac_valle['llave'] = nac_valle['ID'].astype(str) + nac_valle['fecha_nacimiento'].astype(str)
nac_cali['llave'] = nac_cali['ID'].astype(str) + nac_cali['fecha_nacimiento'].astype(str)

# Buscar duplicados en 'llave' en nac_valle
indices_duplicados_valle = nac_valle[nac_valle['llave'].isin(nac_cali['llave'])].index


Int64Index([], dtype='int64')

NO SE ENCONTRARON NACIMIENTOS REPETIDOS. PROCEDEMOS A JUNTARLOS

In [193]:
nac_cali.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11725 entries, 42 to 402250
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Municipio Nacimiento            11725 non-null  object        
 1   Area Nacimiento                 11725 non-null  object        
 2   Sitio Parto                     11725 non-null  object        
 3   IPS                             11721 non-null  object        
 4   Sexo                            11725 non-null  object        
 5   Peso                            11725 non-null  float64       
 6   Talla                           11725 non-null  float64       
 7   Fecha Nacimiento                11725 non-null  datetime64[ns]
 8   Parto Atendido Por              11725 non-null  object        
 9   Tiempo Gestación                11725 non-null  float64       
 10  Número Consultas Prenatales     11724 non-null  float64       
 11  

In [207]:
nac_valle_total = pd.concat([nac_cali, nac_valle])

In [203]:
nac_valle_total.shape

(19407, 33)

In [209]:
nac_valle_total = nac_valle_total.sort_values(by='Fecha Nacimiento')
nac_valle_total['Año'] = nac_valle_total['Fecha Nacimiento'].dt.year
nac_valle_total = df_final.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [210]:
nac_valle_total.to_csv('nac_valle_total_V2.csv', index=False)

# CAPITULO 2. PREPROCESAMIENTO PARA MORTALIDAD

## 2.1. CARGA DE DATOS DEFUNCIONES CALI Y VALLE

In [212]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [213]:
directorio = '/content/drive/My Drive/Datos_BPN2023'

In [214]:
dfs2 = []

# Directorio donde se encuentran los archivos de Excel
anyo_ini=2011
for filename in os.listdir(directorio):
    if filename.startswith("MORTAL") and filename.endswith(('xls', 'xlsx')):  # Filtrar si el archivo es de nacimiento

        df = pd.read_excel(os.path.join(directorio, filename))
        df['anyo']=anyo_ini
        print(filename)
        dfs2.append(df)
        anyo_ini=anyo_ini+1

MORTAL 2018.xlsx
MORTAL 2022.xlsx
MORTAL 2017.xlsx
MORTAL 2019.xlsx
MORTAL 2021.xlsx
MORTAL 2016.xlsx
MORTAL 2020.xlsx
MORTAL 2014.xlsx
MORTAL 2012.xlsx
MORTAL 2011.xlsx
MORTAL 2013.xlsx
MORTAL 2015.xlsx


In [215]:
dfs_mort=list(map(lambda x: x[['Fecha Defunción','Sexo','Tipo Edad Fallecido','Edad Fallecido','Probable Manera Muerte','Tiempo Gestación','Peso','Tipo Documento Madre','Número Documento Madre','Municipio Residencia', 'Municipio Defunción','Diagnóstico A','anyo']], dfs2))

In [216]:
df_mort = pd.concat(dfs_mort, ignore_index=True)

In [217]:
mort_valle=pd.read_excel('/content/drive/My Drive/Datos_BPN2023/BD_Def_2011_2021_HABC_v2_(1).xlsx')

In [218]:
def_cali = df_mort[['Fecha Defunción', 'Sexo', 'Tipo Edad Fallecido', 'Edad Fallecido',
       'Probable Manera Muerte', 'Tiempo Gestación', 'Peso',
       'Tipo Documento Madre', 'Número Documento Madre',
       'Municipio Residencia', 'Municipio Defunción', 'Diagnóstico A']].copy()

In [223]:
def_valle = mort_valle[['FECHA_DEF' , 'SEXO' , 'Tipo Edad Fallecido' , 'Edad Fallecido' ,
                      'PMAN_MUER' , 'T_GES' , 'PESO_NAC' , 'DOC_IDM' , 'NRO_DOCM' ,'CODMUNRE','COD_MUNIC', 'N_DIR1']].copy()

In [221]:
df_mapeo = pd.read_excel("/content/drive/My Drive/Datos_BPN2023/codificacion variables mortalidad.xlsx")

In [224]:
mapeo_dict = {}
for columna in df_mapeo['name'].unique():
    sub_df = df_mapeo[df_mapeo['name'] == columna].copy()
    mapeo_dict[columna] = dict(zip(sub_df['numero'], sub_df['categoria']))
for columna, mapeo in mapeo_dict.items():
    def_valle.loc[:,columna] = def_valle.loc[:,columna].map(mapeo)

In [225]:
def_valle.columns = ['Fecha Defunción', 'Sexo', 'Tipo Edad Fallecido', 'Edad Fallecido',
       'Probable Manera Muerte', 'Tiempo Gestación', 'Peso',
       'Tipo Documento Madre', 'Número Documento Madre',
       'Municipio Residencia', 'Municipio Defunción', 'Diagnóstico A']

In [226]:
print(def_valle.shape)
print(def_cali.shape)

(144067, 12)
(231643, 12)


## 2.2. FILTROS PARA LLEGAR A LAS DEFUNCIONES OBJETIVOS

FILTRO 1: APROVECHANDO LA VARIABLE TIPO EDAD MUERTE BUSCAMOS FILTRAR PRIMERO TODAS LAS MUERTES INFANTILES MENORES A UN AÑO.

In [227]:
def_valle['Tipo Edad Fallecido'].value_counts()

DE UN AÑO O  MAS (EN AÑOS CUMPLIDOS)    141119
MENOR DE 1 MES (EN DÍAS)                  1419
MENOR DE 1 AÑO (EN MESES)                  906
MENOR DE 1 DÍA (EN HORAS)                  425
MENOR DE 1 HORA (EN MINUTOS)               181
Name: Tipo Edad Fallecido, dtype: int64

In [252]:
def_cali['Tipo Edad Fallecido'].value_counts()

DE UN AÑO O  MAS (EN AÑOS CUMPLIDOS)    191405
MENOR DE 1 MES (EN DÍAS)                  2472
MENOR DE 1 AÑO (EN MESES)                 1587
MENOR DE 1 DÍA (EN HORAS)                  616
MENOR DE 1 HORA (EN MINUTOS)               255
Name: Tipo Edad Fallecido, dtype: int64

Con la siguiente impresión corroboraremos que una vez quitados los valores de la variable Tipo de Edad Fallecido 'UN AÑO O MÁS' no se queden descartados valores de menores a un año por error. Es decir corroboramos la calidad del filtro.

In [239]:
print(def_valle[def_valle['Tipo Edad Fallecido'] == 'DE UN AÑO O  MAS (EN AÑOS CUMPLIDOS)']['Tiempo Gestación'].isna().value_counts())
print(def_valle[def_valle['Tipo Edad Fallecido'] == 'DE UN AÑO O  MAS (EN AÑOS CUMPLIDOS)']['Peso'].isna().value_counts())
print(def_valle[def_valle['Tipo Edad Fallecido'] == 'DE UN AÑO O  MAS (EN AÑOS CUMPLIDOS)']['Tipo Documento Madre'].isna().value_counts())

True    141119
Name: Tiempo Gestación, dtype: int64
False    93234
True     47885
Name: Peso, dtype: int64
True     141109
False        10
Name: Tipo Documento Madre, dtype: int64


In [245]:
print(def_cali[def_cali['Tipo Edad Fallecido'] == 'DE UN AÑO O  MAS (EN AÑOS CUMPLIDOS)']['Tiempo Gestación'].isna().value_counts())
print(def_cali[def_cali['Tipo Edad Fallecido'] == 'DE UN AÑO O  MAS (EN AÑOS CUMPLIDOS)']['Peso'].isna().value_counts())
print(def_cali[def_cali['Tipo Edad Fallecido'] == 'DE UN AÑO O  MAS (EN AÑOS CUMPLIDOS)']['Tipo Documento Madre'].isna().value_counts())

True    191405
Name: Tiempo Gestación, dtype: int64
True    191405
Name: Peso, dtype: int64
True    191405
Name: Tipo Documento Madre, dtype: int64


Todos deberían dar true porque se cuentan valores vacios. como vimos que en valle, el peso hay gran cantidad de no vacios. imprimimos un `".describe"`: Para darnos cuenta que aunque no son vacíos siguen siendo 0.0. En conclusión, hacer dicho filtro para llegar a los menores a un año es correcto.

In [253]:
f_menoraño_cali = def_cali['Tipo Edad Fallecido'] != 'DE UN AÑO O  MAS (EN AÑOS CUMPLIDOS)'
f_menoraño_valle = def_valle['Tipo Edad Fallecido'] != 'DE UN AÑO O  MAS (EN AÑOS CUMPLIDOS)'

In [256]:
(def_valle['Tipo Edad Fallecido'] != 'DE UN AÑO O  MAS (EN AÑOS CUMPLIDOS)').value_counts()

False    141119
True       2948
Name: Tipo Edad Fallecido, dtype: int64

In [254]:
print(f'Las defunciones de menores a un año en Cali son:',f_menoraño_cali.sum())
print(f'Las defunciones de menores a un año en Valle son:',f_menoraño_valle.sum())

Las defunciones de menores a un año en Cali son: 40238
Las defunciones de menores a un año en Valle son: 2948


Es posible que en el valle del cauca exista un subregistro debido a que la comparación con Cali, aunque debería ser menor, es extremadamente diferente. ❗

### FILTRO MI

In [260]:
def_inf_cali = def_cali[f_menoraño_cali]
def_inf_valle = def_valle[f_menoraño_valle]

In [305]:
print(def_inf_cali.shape)
print(def_inf_valle.shape)

(40238, 12)
(2948, 12)


 #### IGNORAR:

In [271]:
print(f'Médidas de tendencia del tiempo de gestación y el peso en cali: \n ', def_inf_cali[['Tiempo Gestación', 'Peso']].describe())
print(f'Médidas de tendencia del tiempo de gestación y el peso en valle: \n ', def_inf_valle[['Tiempo Gestación', 'Peso']].describe())

Médidas de tendencia del tiempo de gestación y el peso en cali: 
         Tiempo Gestación          Peso
count      30869.000000  30757.000000
mean          15.523632    524.045876
std           10.709358    940.478206
min            0.000000      0.000000
25%            7.000000     12.000000
50%           10.000000     50.000000
75%           23.000000    530.000000
max           42.000000   9000.000000
Médidas de tendencia del tiempo de gestación y el peso en valle: 
         Tiempo Gestación         Peso
count       2931.000000  2948.000000
mean          38.920505  2546.020692
std           21.493697  2381.200698
min            1.000000     0.000000
25%           27.000000   979.000000
50%           34.000000  2040.000000
75%           39.000000  3000.000000
max           99.000000  9999.000000


VEMOS LA CALIDAD DEL DATO DE LAS VARIABLES A FILTRAR: LOS ERRORES EN ESTAS VARIABLES PUEDEN SUGERIR HACER IMPUTACIÓN O QUITARLOS

In [275]:
print(((def_inf_cali['Peso'] == 0) | (def_inf_cali['Tiempo Gestación'] == 0)).value_counts())
print(((def_inf_valle['Peso'] == 0) | (def_inf_valle['Tiempo Gestación'] == 0)).value_counts())

False    40194
True        44
dtype: int64
False    2931
True       17
dtype: int64


In [281]:
print(((def_inf_cali['Peso'] >= 9000) | (def_inf_cali['Tiempo Gestación'] == 99)).value_counts())
print(((def_inf_valle['Peso'] >= 9000) | (def_inf_valle['Tiempo Gestación'] == 99)).value_counts())

False    40235
True         3
dtype: int64
False    2588
True      360
dtype: int64


In [285]:
print((((def_inf_cali['Peso'] == 0) | (def_inf_cali['Peso'] >= 9000)| (def_inf_cali['Peso'].isna()) ) |
((def_inf_cali['Tiempo Gestación'] == 0) | (def_inf_cali['Tiempo Gestación'] == 99)| (def_inf_cali['Tiempo Gestación'].isna()) )).value_counts())

False    24971
True     15267
dtype: int64


In [286]:
print((((def_inf_valle['Peso'] == 0) | (def_inf_valle['Peso'] >= 9000)| (def_inf_valle['Peso'].isna()) ) |
((def_inf_valle['Tiempo Gestación'] == 0) | (def_inf_valle['Tiempo Gestación'] == 99)| (def_inf_valle['Tiempo Gestación'].isna()) )).value_counts())

False    2571
True      377
dtype: int64


In [296]:
f_bpnt_cali = (def_inf_cali['Peso'] < 2500) & (def_inf_cali['Tiempo Gestación'] >= 37)
f_bpnt_valle = (def_inf_valle['Peso'] < 2500) & (def_inf_valle['Tiempo Gestación'] >= 37)

In [293]:
print(f_bpnt_cali.value_counts())
print(f_bpnt_valle.value_counts())

False    39781
True       457
dtype: int64
False    2742
True      206
dtype: int64


### FILTRO BPNT:

In [299]:
def_bpnt_cali = def_inf_cali[f_bpnt_cali]
def_bpnt_valle = def_inf_valle[f_bpnt_valle]

In [301]:
print(def_bpnt_cali.shape)
print(def_bpnt_valle.shape)

(457, 12)
(206, 12)


In [308]:
((def_bpnt_cali['Número Documento Madre'] == 99999999999)|
 (def_bpnt_cali['Número Documento Madre'].isna())|
 (def_bpnt_cali['Número Documento Madre'] == 0)).value_counts()

False    444
True      13
Name: Número Documento Madre, dtype: int64

In [309]:
((def_bpnt_valle['Número Documento Madre'] == 99999999999)|
 (def_bpnt_valle['Número Documento Madre'].isna())|
 (def_bpnt_valle['Número Documento Madre'] == 0)).value_counts()

False    196
True      10
Name: Número Documento Madre, dtype: int64

In [315]:
f_idna_def_cali= ((def_bpnt_cali['Número Documento Madre'] == 99999999999)|
 (def_bpnt_cali['Número Documento Madre'].isna())|
 (def_bpnt_cali['Número Documento Madre'] == 0))

f_idna_def_valle=((def_bpnt_valle['Número Documento Madre'] == 99999999999)|
 (def_bpnt_valle['Número Documento Madre'].isna())|
 (def_bpnt_valle['Número Documento Madre'] == 0))

In [316]:
def_bpnt_cali = def_bpnt_cali[~f_idna_def_cali].copy()
def_bpnt_valle = def_bpnt_valle[~f_idna_def_valle].copy()

In [319]:
print(def_bpnt_cali.shape)
print(def_bpnt_valle.shape)

(444, 12)
(196, 12)


In [369]:
def_bpnt_cali['Número Documento Madre'].astype(str).isin(def_bpnt_valle['Número Documento Madre'].astype(int).astype(str)).value_counts()

False    363
True      81
Name: Número Documento Madre, dtype: int64

In [334]:
def_bpnt_cali['Número Documento Madre'].astype(str).isin(def_bpnt_valle['Número Documento Madre'].astype(int).astype(str)).index

Int64Index([  2057,   2151,   2268,   2813,   3354,   4332,   4505,   4649,
              4726,   6838,
            ...
            226236, 227186, 227411, 228007, 228586, 229295, 229390, 230104,
            230642, 231398],
           dtype='int64', length=444)

In [356]:
def_bpnt_valle['Número Documento Madre'].astype(int).astype(str).isin(def_bpnt_cali['Número Documento Madre'].astype(str)).index

Int64Index([  1104,   4189,   5625,   6980,   7554,   9477,  11317,  11534,
             11944,  11952,
            ...
            130706, 131275, 131345, 131583, 131985, 135902, 137399, 140392,
            141513, 143922],
           dtype='int64', length=196)

In [362]:
def_bpnt_valle.loc[1104,]

Fecha Defunción                 2011-02-05 00:00:00
Sexo                                              1
Tipo Edad Fallecido       MENOR DE 1 AÑO (EN MESES)
Edad Fallecido                                    4
Probable Manera Muerte                      NATURAL
Tiempo Gestación                               99.0
Peso                                         1950.0
Tipo Documento Madre                              3
Número Documento Madre                   38671277.0
Municipio Residencia                           CALI
Municipio Defunción                            CALI
Diagnóstico A                        FALLA CARDIACA
Name: 1104, dtype: object

In [363]:
def_bpnt_valle.loc[def_bpnt_valle['Número Documento Madre'].astype(int) == 38671277,]

,Fecha Defunción,Sexo,Tipo Edad Fallecido,Edad Fallecido,Probable Manera Muerte,Tiempo Gestación,Peso,Tipo Documento Madre,Número Documento Madre,Municipio Residencia,Municipio Defunción,Diagnóstico A
1104,2011-02-05 00:00:00,1,MENOR DE 1 AÑO (EN MESES),4,NATURAL,99.0,1950.0,3,38671277.0,CALI,CALI,FALLA CARDIACA


In [365]:
def_bpnt_cali.loc[def_bpnt_cali['Número Documento Madre'].astype(str) == '38671277',]

,Fecha Defunción,Sexo,Tipo Edad Fallecido,Edad Fallecido,Probable Manera Muerte,Tiempo Gestación,Peso,Tipo Documento Madre,Número Documento Madre,Municipio Residencia,Municipio Defunción,Diagnóstico A


In [ ]:
# LA COMPARACIÓN DE CÉDULAS REPETIDAS DA COMO RESULTADO CIERTOS INDICES PERO NO LO PUDIMOS CORROBORAR.